In [1]:
import socket
import hmac
import numpy as np
import cv2



def divide_text(text, block_size_bits=16, X=None):
    text = text.encode('utf-8')
    block_size_bytes = block_size_bits // 8
    blocks = [text[i:i + block_size_bytes] for i in range(0, len(text), block_size_bytes)]
    
    # Padding to ensure the number of blocks is a multiple of X.shape[0]
    if X is not None and len(blocks) % X.shape[0] != 0:
        padding_size = X.shape[0] - len(blocks) % X.shape[0]
        blocks.extend([b''] * padding_size)
    
    return np.array(blocks)

def mac_for_block(blocks, key, X, Y):
    res = {}
    for msg_index in range(X.shape[0]):
        res[msg_index] = blocks[msg_index]
    
    for tag_index in range(X.shape[1]):
        selected_blocks = blocks[X[:, tag_index] == 1]
        if selected_blocks.size > 0:
            data = b''.join(selected_blocks)
            target_index = np.where(Y[:, tag_index] == 1)[0][0]
            res[target_index] = blocks[target_index] + hmac.new(key, data, digestmod='sha384').digest()
    
    return res

def send_msg(FN, msg, sock, dest):
    sock.sendto(FN.to_bytes(4, 'big') + msg, dest)

def tx(text, sock, ip, port, X, Y, block_size_bits=128, attack=[]):
    blocks = divide_text(text, block_size_bits=block_size_bits, X=X)
    SN = 1
    total_mac_length = 0
    
    for i in range(0, len(blocks), X.shape[0]):
        block = mac_for_block(blocks[i:i + X.shape[0]], key=b"key", X=X, Y=Y)
        total_mac_length += X.shape[1] * 384 // 8
        
        for msg in block.values():
            if FN in attack:
                FN += 1
                continue
            send_msg(FN, msg, sock, (ip, port))
            FN += 1
            time.sleep(0.001)
    
    sock.sendto(b'END', (ip, port))
    sock.close()
    return total_mac_length

# Matrix Definitions (X and Y)
X = np.array([[1, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
              [1, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
              [1, 0, 0, 0, 0, 0, 0, 0, 1],  # m3
              [0, 1, 0, 0, 0, 0, 1, 0, 0],  # m4
              [0, 1, 0, 0, 0, 0, 0, 1, 0],  # m5
              [0, 1, 0, 0, 0, 0, 0, 0, 1],  # m6
              [0, 0, 1, 0, 0, 0, 1, 0, 0],  # m7
              [0, 0, 1, 0, 0, 0, 0, 1, 0],  # m8
              [0, 0, 1, 0, 0, 0, 0, 0, 1],  # m9
              [0, 0, 0, 1, 0, 0, 1, 0, 0],  # m10
              [0, 0, 0, 1, 0, 0, 0, 1, 0],  # m11
              [0, 0, 0, 1, 0, 0, 0, 0, 1],  # m12
              [0, 0, 0, 0, 1, 0, 1, 0, 0],  # m13
              [0, 0, 0, 0, 1, 0, 0, 1, 0],  # m14
              [0, 0, 0, 0, 1, 0, 0, 0, 1],  # m15
              [0, 0, 0, 0, 0, 1, 1, 0, 0],  # m16
              [0, 0, 0, 0, 0, 1, 0, 1, 0],  # m17
              [0, 0, 0, 0, 0, 1, 0, 0, 1]]) # m18

Y = np.array([[0, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
              [0, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
              [1, 0, 0, 0, 0, 0, 0, 0, 0],  # m3
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m4
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m5
              [0, 1, 0, 0, 0, 0, 0, 0, 0],  # m6
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m7
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m8
              [0, 0, 1, 0, 0, 0, 0, 0, 0],  # m9
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m10
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m11
              [0, 0, 0, 1, 0, 0, 0, 0, 0],  # m12
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m13
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m14
              [0, 0, 0, 0, 1, 0, 0, 0, 0],  # m15
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m16
              [0, 0, 0, 0, 0, 1, 0, 0, 0],  # m17
              [0, 0, 0, 0, 0, 0, 0, 0, 1]]) # m18

def send_frame(frame, sock, ip, port, key, frame_index):
    # Encode frame to JPEG
    _, encoded_frame = cv2.imencode('.jpg', frame)
    data = encoded_frame.tobytes()
    
    # Split data into chunks
    chunk_size = 1024 * X.shape[0] # Define the size of each chunk
    for i in range(0, len(data)//chunk_size):
        mac = hmac.new(key, data[i*chunk_size:(i+1)*chunk_size], digestmod='sha384').digest()
        message = frame_index.to_bytes(4, 'big') + i.to_bytes(4, 'big') + data[i*chunk_size:(i+1)*chunk_size] + mac

    
    for i, chunk in enumerate(chunks):
        mac = hmac.new(key, chunk, digestmod='sha384').digest()
        if i%40==1:
            mac = hmac.new(key+b'kkasd', chunk, digestmod='sha384').digest()
        message = frame_index.to_bytes(4, 'big') + i.to_bytes(4, 'big') + chunk + mac  # Frame index, chunk index, chunk, and MAC
        sock.sendto(message, (ip, port))
    sock.sendto(b'END', (ip, port))  # indicate end of frame transmission

# Example usage
ip = "0.0.0.0"
port = 23422
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
key = b'secret_key'

cap = cv2.VideoCapture(0)  # Capture video from the default camera

frame_index = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    data = send_frame(frame, sock, ip, port, key, frame_index)
    frame_index += 1

    # Display the frame locally
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()
sock.close()


[ WARN:0@0.285] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


NameError: name 'chunks' is not defined

In [2]:

import cryptography.hazmat.primitives.padding as padding
import numpy as np
import socket
import hmac
import pprint


class UDP_TX:
    def __init__(self, IP:str ='0.0.0.0', PORT:int = 23422, X = np.eye(10), Y = np.eye(10), chunk_size_Byte=128, KEY=b"key", digestmod='sha384'):
        self.IP = IP
        self.PORT = PORT
        self.X = X
        self.Y = Y
        self.chunk_size_Byte = chunk_size_Byte
        self.KEY = KEY
        self.digestmod = digestmod

    # Padding using PKCS7
    def pad(self,data, chunk_size_Byte = None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        
        padder = padding.PKCS7(chunk_size_Byte*8).padder()
        padded_data = padder.update(data) + padder.finalize()
        return padded_data


    def divide_Bytes_and_pad(self,Bytes, chunk_size_Byte=None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        
        chunks = [Bytes[i:i + chunk_size_Byte] for i in range(0, len(Bytes), chunk_size_Byte)]
        if len(chunks[-1]) != chunk_size_Byte:
            chunks[-1] = self.pad(chunks[-1], chunk_size_Byte)
        return chunks

    def chucks_to_nD_arrangments(self,chunks,chunk_size_Byte = None, X= None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        if X is None:
            X = self.X

        nD_arrangments = [chunks[i:i + X.shape[0]] for i in range(0, len(chunks), X.shape[0])]
        if len(nD_arrangments[-1]) != X.shape[0]:
            for i in range( X.shape[0] - len(nD_arrangments[-1]))  :
                nD_arrangments[-1].append(self.pad(b'',chunk_size_Byte=chunk_size_Byte))
        return np.array(nD_arrangments)

    def mac_for_page(self,page, key = None, X = None, Y = None):
        if key is None:
            key = self.KEY
        if X is None:
            X = self.X
        if Y is None:
            Y = self.Y
        
        res = {}
        for msg_index in range(X.shape[0]):
            res[msg_index] = page[msg_index]
        
        for tag_index in range(X.shape[1]):
            selected_blocks = page[X[:, tag_index] == 1]
            if selected_blocks.size > 0:
                data = b''.join(selected_blocks)
                target_index = np.where(Y[:, tag_index] == 1)[0][0]
                res[target_index] = page[target_index] + hmac.new(key, data, digestmod=self.digestmod).digest()
        return res

    def send_msg(self,SN, msg, sock, dest):
        sock.sendto(SN.to_bytes(4, 'big') + msg, dest)


    def transmit(self, Bytes, attack:list=[]):

        SN = 0
        cnt = 0
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            chunks = self.divide_Bytes_and_pad(Bytes = Bytes, chunk_size_Byte= self.chunk_size_Byte)
            pages = self.chucks_to_nD_arrangments(chunks = chunks, chunk_size_Byte= self.chunk_size_Byte, X =  self.X)

            for i in range(len(pages)):
                page = self.mac_for_page(page=pages[i], key= self.KEY, X= self.X, Y= self.Y)
                for msg in page.values():
                    if SN in attack:
                        SN += 1
                        continue
                    send_msg(SN, msg, sock, (self.IP,  self.PORT))
                    cnt += 1
                    SN += 1
            ### send the end message this is not secure but it is just for testing
            sock.sendto(b'END', ( self.IP,  self.PORT))
        return cnt

# unit test




IP = "0.0.0.0"
PORT = 23422

X = np.eye(3)
Y = np.eye(3)

chunk_size_Byte = 7

key = b"key"

upd_tx = UDP_TX(IP, PORT, X, Y, chunk_size_Byte, key)

data = b'This test shows 2D integrity check is better than blockwise integrity.'

# testing the divide_Bytes_and_pad function

print("Dividing the Byte data every to chunk_size_Bytes and pad the last chunck if needed",upd_tx.divide_Bytes_and_pad(data, chunk_size_Byte=chunk_size_Byte))

# testing the chucks_to_nD_arrangments function witht X.shape[0] and chunk_size
chunks = upd_tx.divide_Bytes_and_pad(data, chunk_size_Byte=chunk_size_Byte)
print(upd_tx.chucks_to_nD_arrangments(chunks, chunk_size_Byte, X))

# testing the mac_for_page function
pages = upd_tx.chucks_to_nD_arrangments(chunks, chunk_size_Byte, X)
pprint.pprint(upd_tx.mac_for_page(pages[0], key, X, Y))

# at this point after calculating the MAC for each page, we can send the data
# iterate through the chunks (msg) in every page and send it to the destination
# the function send_msg is responsible for sending the data to the destination
# the function transmit input bytes and create pages with padded chunks  and 
# then iterates through the all the chunks in the pages and send them to the destination

# testing the transmit function
print("Number of messages sent: ", upd_tx.transmit(data))
# testing the 
print("Number of messages sent under attack: ", upd_tx.transmit(data, attack=[1,3]))






Dividing the Byte data every to chunk_size_Bytes and pad the last chunck if needed [b'This te', b'st show', b's 2D in', b'tegrity', b' check ', b'is bett', b'er than', b' blockw', b'ise int', b'egrity.']
[[b'This te' b'st show' b's 2D in']
 [b'tegrity' b' check ' b'is bett']
 [b'er than' b' blockw' b'ise int']
 [b'egrity.' b'\x07\x07\x07\x07\x07\x07\x07'
  b'\x07\x07\x07\x07\x07\x07\x07']]
{0: b'This te\xd2\x1b\x97n\xe9\xf9\x10\xc8f\xd6\xac-e\x91\xfc\xa8\xddpt\xb6\x1b'
    b'>\x98f%\x88? F\\,j\xde-v3"]\xcf|\x1a\xcf\xcbu\x19\xcfc3',
 1: b'st show\n\xad46@\x9c\xce\x82\x1a\xa0\x17\xb8\xed\xd8N\x1d\xd2'
    b'\x92\xc8Z\xb4\xae\x90\xa4*\xb6\x1f\xbb\xedAF\n\x11\xd4@\x9e\xdbeDC\x80'
    b'\xf3\x13\x00R\xdbM\xfd',
 2: b's 2D in\x17\\\xc3\x00:2\x08y\x1ce\xf2\xc9\xb5\xb4/\x96\x14\xc5\xee\x8eS'
    b'\xd4\xb7\x8c\xf9B\xff\xb2\xc2s\xa97\xa7q\xfc\xefU<\x82\xf9\xf8mC)\x0e'
    b'\xc7\x0cf'}
Number of messages sent:  12
Number of messages sent under attack:  10


In [16]:

# define a Buffer with the size of chunck_size_Byte * X.shape[0] * number_of_pages
from collections import OrderedDict
import numpy as np
import time


# Buffer size is in pages where each page is a list of X.shape[0]*chuncks and every chunck is of size chunk_size_Byte
# the buffer is a list of pages
# every page is a dict of chuncks in the format of SN:(msg,mac)
# every chunck is a list of bytes
# the buffer is updated when a new message is received
# the buffer is used to reorder the messages

# page zero keep a timer of the latest time the page is updated
# this is used to remove the old page if a SN comes and the time from the last update is more than timeout
class Buffer:

    def __init__(self, X, Y, BUFFER_SIZE_IN_PAGES = 10, TIMEOUT_SECOND = 1,  warnings = True):
    
        # defining the buffer with maxlen of number_of_pages
        # some chunks are padded
        self.X = X
        self.Y = Y

        self.BUFFER_SIZE_IN_PAGES = BUFFER_SIZE_IN_PAGES
        self.BUFFER = []

        #initialize the buffer with empty page 
        for i in range(0,BUFFER_SIZE_IN_PAGES):
            self.add_page()
 

        self.TIMEOUT_SECOND = TIMEOUT_SECOND
        self.PAGE_ZERO_LAST_UPDATE = time.time()

        self.MIN_SN = 0

        self.warnings = warnings


    

    def get_min_allowed_SN(self):
        return self.MIN_SN
    def get_max_allowed_SN(self):
        return self.MIN_SN - (self.MIN_SN % self.X.shape[0]) + self.X.shape[0]*len(self.BUFFER) -1
    

    def sort_SN_in_page(self, page):
        return {k: v for k, v in sorted(page.items(), key=lambda item: item[0])}

    def add_page(self):
        if len(self.BUFFER)  < self.BUFFER_SIZE_IN_PAGES:
            self.BUFFER.append({})
            return True
        if self.warnings:
            print("The buffer is full, increase the buffer size or this might be an attack to the buffer.")
        return False
    
    def pop_page(self, page_index):
        if page_index > len(self.BUFFER) or page_index < 0:
            if self.warnings:
                print(f"Page {page_index} is out of range, the buffer size is {len(self.BUFFER)}")
            return None
        if page_index == 0:
            self.PAGE_ZERO_LAST_UPDATE = time.time()
            self.MIN_SN += self.X.shape[0]

        temp = self.sort_SN_in_page(self.BUFFER.pop(page_index))
        self.add_page()
        return temp
    

    def get_page_index_by_SN(self, SN):
        return  ((SN-self.MIN_SN) // self.X.shape[0])%self.BUFFER_SIZE_IN_PAGES
    

    def is_page_full(self, page_index):
        return len(self.BUFFER[page_index]) == self.X.shape[0]

    # three possible return values None, (page,None), (page, page)
    def add_msg_to_page(self, SN, msg, mac = b''):
        l, r = self.get_min_allowed_SN(), self.get_max_allowed_SN()
        if  SN < l or SN > r and self.warnings:
            print(f"SN {SN} is out of range [{l,r}] (Buffer full), increase the buffer size or this might be an attack to the buffer.")
            if time.time() - self.PAGE_ZERO_LAST_UPDATE < self.TIMEOUT_SECOND:
                print(f" The message SN: {SN} is dropped. The buffer is full and the page zero will be kept until {self.TIMEOUT_SECOND -time.time() + self.PAGE_ZERO_LAST_UPDATE} more seconds")
                return None
            else:
                min_sn = self.get_min_allowed_SN()
                res = self.pop_page(0)
                temp = self.add_msg_to_page(SN, msg)
                return res ,temp, min_sn 
            

        page_index = self.get_page_index_by_SN(SN)

        if SN in self.BUFFER[page_index] and self.warnings:
            return "Message already exists in the buffer! Replay attack?"
        
        if page_index == 0: 
            self.PAGE_ZERO_LAST_UPDATE = time.time()
        
        self.BUFFER[page_index][SN] = (msg, mac)
          
        if self.is_page_full(page_index):
            return self.pop_page(page_index)

        return None
    
    def print_buffer(self):
        print(f"{self.MIN_SN} Buffer:", self.BUFFER)
    

# unit test for the buffer

X = np.eye(3)
Y = np.eye(3)

buffer = Buffer(X, Y, BUFFER_SIZE_IN_PAGES = 3, TIMEOUT_SECOND = 1,  warnings = True)

print(buffer.get_min_allowed_SN())
print(buffer.get_max_allowed_SN())

print(buffer.add_page())

print(buffer.get_page_index_by_SN(0))
print(buffer.get_page_index_by_SN(1))

print(buffer.is_page_full(0))

print(buffer.add_msg_to_page(7, 'msg7'))
print(buffer.add_msg_to_page(3, 'msg3'))
print(buffer.add_msg_to_page(0, 'msg0'))
print(buffer.add_msg_to_page(1, 'msg1'))
print(buffer.add_msg_to_page(2, 'msg2'))

print(buffer.add_msg_to_page(4, 'msg4'))
print(buffer.add_msg_to_page(5, 'msg5'))

print(buffer.add_msg_to_page(6, 'msg6'))

print(buffer.add_msg_to_page(8, 'msg8'))

print(buffer.add_msg_to_page(-1, 'msg0'))

print(buffer.add_msg_to_page(100, 'msg0'))
time.sleep(1.1)
print(buffer.add_msg_to_page(20, 'msg0'))
time.sleep(1.1)
print(buffer.add_msg_to_page(100, 'msg0'))

0
8
The buffer is full, increase the buffer size or this might be an attack to the buffer.
False
0
0
False
None
None
None
None
{0: ('msg0', b''), 1: ('msg1', b''), 2: ('msg2', b'')}
None
{3: ('msg3', b''), 4: ('msg4', b''), 5: ('msg5', b'')}
None
{6: ('msg6', b''), 7: ('msg7', b''), 8: ('msg8', b'')}
SN -1 is out of range [(9, 17)] (Buffer full), increase the buffer size or this might be an attack to the buffer.
 The message SN: -1 is dropped. The buffer is full and the page zero will be kept until 0.9999692440032959 more seconds
None
SN 100 is out of range [(9, 17)] (Buffer full), increase the buffer size or this might be an attack to the buffer.
 The message SN: 100 is dropped. The buffer is full and the page zero will be kept until 0.9999396800994873 more seconds
None
SN 20 is out of range [(9, 17)] (Buffer full), increase the buffer size or this might be an attack to the buffer.
({}, None, 9)
SN 100 is out of range [(12, 20)] (Buffer full), increase the buffer size or this might be

In [18]:
    






class UDP_RX:
    def __init__(self,buffer= None, IP:str ='0.0.0.0', PORT:int = 23422, X = np.eye(3), Y = np.eye(3),  chunk_size_Byte=128, KEY=b"key", digestmod='sha384', BUFFER_SIZE_IN_PAGES = 10):
        self.IP = IP
        self.PORT = PORT
        self.X = X
        self.Y = Y
        self.chunk_size_Byte = chunk_size_Byte
        self.KEY = KEY
        self.digestmod = digestmod
        self.HAMC_SIZE = hmac.new(KEY, b'', digestmod=digestmod).digest_size
        self.BUFFER_SIZE_IN_PAGES = BUFFER_SIZE_IN_PAGES

        if buffer is None:
            self.BUFFER = Buffer( X, Y, chunk_size_Byte, BUFFER_SIZE_IN_PAGES)
        else:
            self.BUFFER = buffer
        
    def parse_msg(self, data):
        SN = int.from_bytes(data[:4], 'big')
        if np.sum(self.Y[SN % self.X.shape[0] - 1]):
            chunk_data = data[4:-self.HAMC_SIZE]
            mac = data[-self.HAMC_SIZE:]
        else:
            chunk_data = data[4:]
            mac = b''
        return SN, chunk_data, mac
    
    def veify_page(self, page:dict,res:dict, key = None, X=None, Y= None):
        if key is None:
            key = self.KEY
        if X is None:
            X = self.X
        if Y is None:
            Y = self.Y

        page_array = np.array(list(page.values()))
        for SN in page.keys():
            res[SN] = np.array([page_array[SN%X.shape[0]][0],0])
        
        for tag_index in range(X.shape[1]):
            selected_blocks = page_array[X[:, tag_index] == 1][:,0]
            if selected_blocks.size > 0:
                corresponding_data = b''.join(selected_blocks)
                recieved_mac = page_array[np.where(Y[:, tag_index] == 1)[0][0]][1]

                if recieved_mac == hmac.new(self.KEY, corresponding_data, digestmod=self.digestmod).digest():
                    # print("Verified", res)
                    for SN in np.array(list(page.keys()))[X[:, tag_index] == 1]:
                        res[SN][1] = int(res[SN][1]) + 1

                else:
                    pass
        return res
    
    def fill_missing_in_page_with_zeros(self, page:dict, SN: int):
        for i in range(SN, SN + self.X.shape[0]):
            if i not in page:
                page[i] = (b'', b'')
        return page

    def receive(self):
        total_res = {}
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            sock.bind((self.IP, self.PORT))
            while True:
                data, addr = sock.recvfrom(4096)
                if data == b'END':
                    break
                SN, chunk_data, mac = self.parse_msg(data)
                res = self.BUFFER.add_msg_to_page(SN, chunk_data, mac)
                # Three stages are possible None, (page,None), (page, page)
                if res is not None:
                    if isinstance(res, dict):
                        total_res = self.veify_page(res, total_res)
                    elif isinstance(res, tuple):
                           
                        page0 = self.fill_missing_in_page_with_zeros(page = res[0], SN = res[2])
                        total_res = self.veify_page(page0, total_res)
                        if res[1] is not None: 
                            total_res = self.veify_page(res[1], total_res)
        return total_res



## unit test for the page verifier
X = np.eye(3)
Y = np.eye(3)


buffer = Buffer(X, Y, BUFFER_SIZE_IN_PAGES = 3, TIMEOUT_SECOND = 0.00001,  warnings = True)
udp_rx = UDP_RX(buffer)
udp_rx.receive()


# x = {0: b'\x00\x00\x00\x00This te\xd2\x1b\x97n\xe9\xf9\x10\xc8f\xd6\xac-e\x91\xfc\xa8\xddpt\xb6\x1b'
#     b'>\x98f%\x88? F\\,j\xde-v3"]\xcf|\x1a\xcf\xcbu\x19\xcfc3',
#  1: b'\x00\x00\x00\x01st show\n\xad46@\x9c\xce\x82\x1a\xa0\x17\xb8\xed\xd8N\x1d\xd2'
#     b'\x92\xc8Z\xb4\xae\x90\xa4*\xb6\x1f\xbb\xedAF\n\x11\xd4@\x9e\xdbeDC\x80'
#     b'\xf3\x13\x00R\xdbM\xfd',
#  2: b'\x00\x00\x00\x02s 2D in\x17\\\xc3\x00:2\x08y\x1ce\xf2\xc9\xb5\xb4/\x96\x14\xc5\xee\x8eS'
#     b'\xd4\xb7\x8c\xf9B\xff\xb2\xc2s\xa97\xa7q\xfc\xefU<\x82\xf9\xf8mC)\x0e'
#     b'\xc7\x0cf'}

# print(buffer.add_msg_to_page(*udp_rx.parse_msg(x[1])))
# print(buffer.add_msg_to_page(*udp_rx.parse_msg(x[0])))
# print(temp:=buffer.add_msg_to_page(*udp_rx.parse_msg(x[2])))

# print(temp)

# print(udp_rx.veify_page(temp, key, X, Y))



SN 9 is out of range [(0, 8)] (Buffer full), increase the buffer size or this might be an attack to the buffer.


{3: array([b'tegrity', b'1'], dtype='|S21'),
 4: array([b' check ', b'1'], dtype='|S21'),
 5: array([b'is bett', b'1'], dtype='|S21'),
 6: array([b'er than', b'1'], dtype='|S21'),
 7: array([b' blockw', b'1'], dtype='|S21'),
 8: array([b'ise int', b'1'], dtype='|S21'),
 0: array([b'This te', b'1'], dtype='|S21'),
 2: array([b'', b'1'], dtype='|S21'),
 1: array([b's 2D in', b'0'], dtype='|S21'),
 9: array([b'egrity.', b'0'], dtype='|S21'),
 10: array([b'\x07\x07\x07\x07\x07\x07\x07', b'1'], dtype='|S21'),
 11: array([b'\x07\x07\x07\x07\x07\x07\x07', b'1'], dtype='|S21')}

In [14]:

import cryptography.hazmat.primitives.padding as padding
import numpy as np
import socket
import hmac
import pprint


class UDP_TX:
    def __init__(self, IP:str ='0.0.0.0', PORT:int = 23422, X = np.eye(10), Y = np.eye(10), chunk_size_Byte=128, KEY=b"key", digestmod='sha384'):
        self.IP = IP
        self.PORT = PORT
        self.X = X
        self.Y = Y
        self.chunk_size_Byte = chunk_size_Byte
        self.KEY = KEY
        self.digestmod = digestmod

    # Padding using PKCS7
    def pad(self,data, chunk_size_Byte = None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        
        padder = padding.PKCS7(chunk_size_Byte*8).padder()
        padded_data = padder.update(data) + padder.finalize()
        return padded_data


    def divide_Bytes_and_pad(self,Bytes, chunk_size_Byte=None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        
        chunks = [Bytes[i:i + chunk_size_Byte] for i in range(0, len(Bytes), chunk_size_Byte)]
        if len(chunks[-1]) != chunk_size_Byte:
            chunks[-1] = self.pad(chunks[-1], chunk_size_Byte)
        return chunks

    def chucks_to_nD_arrangments(self,chunks,chunk_size_Byte = None, X= None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        if X is None:
            X = self.X

        nD_arrangments = [chunks[i:i + X.shape[0]] for i in range(0, len(chunks), X.shape[0])]
        if len(nD_arrangments[-1]) != X.shape[0]:
            for i in range( X.shape[0] - len(nD_arrangments[-1]))  :
                nD_arrangments[-1].append(self.pad(b'',chunk_size_Byte=chunk_size_Byte))
        return np.array(nD_arrangments)

    def mac_for_page(self,page, key = None, X = None, Y = None):
        if key is None:
            key = self.KEY
        if X is None:
            X = self.X
        if Y is None:
            Y = self.Y
        
        res = {}
        for msg_index in range(X.shape[0]):
            res[msg_index] = page[msg_index]
        
        for tag_index in range(X.shape[1]):
            selected_blocks = page[X[:, tag_index] == 1]
            if selected_blocks.size > 0:
                data = b''.join(selected_blocks)
                target_index = np.where(Y[:, tag_index] == 1)[0][0]
                res[target_index] = page[target_index] + hmac.new(key, data, digestmod=self.digestmod).digest()
        return res

    def send_msg(self,SN, msg, sock, dest):
        sock.sendto(SN.to_bytes(4, 'big') + msg, dest)


    def transmit(self, Bytes, attack:list=[]):

        SN = 0
        cnt = 0
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            chunks = self.divide_Bytes_and_pad(Bytes = Bytes, chunk_size_Byte= self.chunk_size_Byte)
            pages = self.chucks_to_nD_arrangments(chunks = chunks, chunk_size_Byte= self.chunk_size_Byte, X =  self.X)

            for i in range(len(pages)):
                page = self.mac_for_page(page=pages[i], key= self.KEY, X= self.X, Y= self.Y)
                for msg in page.values():
                    if SN in attack:
                        SN += 1
                        continue
                    self.send_msg(SN, msg, sock, (self.IP,  self.PORT))
                    cnt += 1
                    SN += 1
            ### send the end message this is not secure but it is just for testing
            sock.sendto(b'END', ( self.IP,  self.PORT))
        return cnt

# unit test




IP = "0.0.0.0"
PORT = 23422

X = np.eye(3)
Y = np.eye(3)

chunk_size_Byte = 7

key = b"key"

upd_tx = UDP_TX(IP, PORT, X, Y, chunk_size_Byte, key)

data = b'This test shows 2D integrity check is better than blockwise integrity.'

# testing the divide_Bytes_and_pad function

print("Dividing the Byte data every to chunk_size_Bytes and pad the last chunck if needed",upd_tx.divide_Bytes_and_pad(data, chunk_size_Byte=chunk_size_Byte))

# testing the chucks_to_nD_arrangments function witht X.shape[0] and chunk_size
chunks = upd_tx.divide_Bytes_and_pad(data, chunk_size_Byte=chunk_size_Byte)
print(upd_tx.chucks_to_nD_arrangments(chunks, chunk_size_Byte, X))

# testing the mac_for_page function
pages = upd_tx.chucks_to_nD_arrangments(chunks, chunk_size_Byte, X)
pprint.pprint(upd_tx.mac_for_page(pages[0], key, X, Y))

# at this point after calculating the MAC for each page, we can send the data
# iterate through the chunks (msg) in every page and send it to the destination
# the function send_msg is responsible for sending the data to the destination
# the function transmit input bytes and create pages with padded chunks  and 
# then iterates through the all the chunks in the pages and send them to the destination

# testing the transmit function
# print("Number of messages sent: ", upd_tx.transmit(data))
# testing the 
print("Number of messages sent under attack: ", upd_tx.transmit(data, attack=[1]))

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 3